<a href="https://colab.research.google.com/github/donggggerr/bert-spam-detection/blob/main/NoChunkingAndRandom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
import pandas as pd
from transformers import BertTokenizer
from tqdm import tqdm
import re
from bs4 import BeautifulSoup

In [ ]:
# Enable tqdm for pandas apply
tqdm.pandas()

In [ ]:
df = pd.read_csv(r"C:\Users\rtgx2\Downloads\Dataset\Preprocessed TREC 2007 Public Corpus Dataset\processed_data.csv")

In [ ]:
df

,label,subject,email_to,email_from,message
0,1,"Generic Cialis, branded quality@",the00@speedy.uwaterloo.ca,"""Tomas Jacobs"" <RickyAmes@aol.com>",Content-Type: text/html;\nContent-Transfer-Enc...
1,0,Typo in /debian/README,debian-mirrors@lists.debian.org,Yan Morin <yan.morin@savoirfairelinux.com>,"Hi, i've just updated from the gulus and I che..."
2,1,authentic viagra,<the00@plg.uwaterloo.ca>,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...","Content-Type: text/plain;\n\tcharset=""iso-8859..."
3,1,Nice talking with ya,opt4@speedy.uwaterloo.ca,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>","Hey Billy, \n\nit was really fun going out the..."
4,1,or trembling; stomach cramps; trouble in sleep...,ktwarwic@speedy.uwaterloo.ca,"""Christi T. Jernigan"" <dcube@totalink.net>",Content-Type: multipart/alternative;\n ...
...,...,...,...,...,...
75414,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>",Content-Type: text/html;\nContent-Transfer-Enc...
75415,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>",Content-Type: text/html;\nContent-Transfer-Enc...
75416,0,"Re: [R] Me again, about the horrible documenta...",Duncan Murdoch <murdoch@stats.uwo.ca>,Philippe Grosjean <phgrosjean@sciviews.org>,"For those who are interested, I just cook a li..."
75417,0,Re: [R] RODBC problem,<r-help@stat.math.ethz.ch>,=?iso-8859-1?Q?Bernhard_Wellh=F6fer?=\n\t<Bern...,"Hello,\n\nas I wrote I call\n\n sqlFetch(chan..."


In [ ]:
print(df['email_from'].head())
print(df['label'].unique())

0                   "Tomas Jacobs" <RickyAmes@aol.com>
1           Yan Morin <yan.morin@savoirfairelinux.com>
2    "Sheila Crenshaw" <7stocknews@tractionmarketin...
3         "Stormy Dempsey" <vqucsmdfgvsg@ruraltek.com>
4           "Christi T. Jernigan" <dcube@totalink.net>
Name: email_from, dtype: object
[1 0]


In [ ]:
df[df['label'] == 1]['email_from'].value_counts().head(10)
# this are spam

email_from
Groupe Desjardins / AccesD <services.de.cartes@scd.desjardins.com>                                                                   2826
Groupe Desjardins / AccesD <servicesprimaire@desjardins.com>                                                                          154
Royal Bank of Canada <securityclient@rbc.com>                                                                                         110
=?ISO-8859-1?Q?S=E9curit=E9?= informatique Mouvement des Caisses\n Desjardins  <services.de.cartes.desjardins@scd.desjardins.com>      92
Groupe Desjardins / AccesD <securiteaccesd@desjardins.com>                                                                             80
"editor@QualityStocks.net" <editor@qsnews.net>                                                                                         63
AFRICAN UNION OFFICE <african_union_office@excite.com>                                                                                 48
"Desjardins Security Se

In [ ]:
df[df['label'] == 1]['email_to'].value_counts().head(10)
# this are spam

email_to
<theorize@plg.uwaterloo.ca>               2069
gnitpick@flax9.uwaterloo.ca               2063
theorize@plg.uwaterloo.ca                 2013
<the00@plg.uwaterloo.ca>                  1965
the00@plg.uwaterloo.ca                    1899
<the00@plg2.math.uwaterloo.ca>            1102
<gnitpick@flax9.uwaterloo.ca>             1075
the00@plg2.math.uwaterloo.ca              1013
gnitpick@speedy.uwaterloo.ca               923
"theorize" <theorize@plg.uwaterloo.ca>     827
Name: count, dtype: int64

In [ ]:
df[df['label'] == 0]['email_from'].value_counts().head(10)
# this are ham

email_from
<alert@broadcast.shareholder.com>                                     756
tridge@samba.org                                                      685
"AccuWeather.com Alert" <inbox@messaging.accuweather.com>             562
CNN Alerts <cnnalerts@mail.cnn.com>                                   366
"Chas Owens" <chas.owens@gmail.com>                                   347
metze@samba.org                                                       335
Prof Brian Ripley <ripley@stats.ox.ac.uk>                             289
chromatic <chromatic@wgz.org>                                         270
"Gerald (Jerry) Carter" <jerry@samba.org>                             267
"=?iso-8859-1?Q?The_Weather_Network?=" <weatherdirect@pelmorex.ca>    255
Name: count, dtype: int64

In [ ]:
df[df['label'] == 0]['email_to'].value_counts().head(10)
# this are ham

email_to
samba-cvs@samba.org                                                        2396
r-help@stat.math.ethz.ch                                                   1885
ktwarwic@flax9.uwaterloo.ca                                                1539
"Speakup is a screen review system for Linux." <speakup@braille.uwo.ca>     979
beginners@perl.org                                                          703
ktwarwic@speedy.uwaterloo.ca                                                536
debian-legal@lists.debian.org                                               489
<ktwarwic@flax9.uwaterloo.ca>                                               442
ktwarwic@flax9.UWATERLOO.CA                                                 396
perl6-internals@perl.org                                                    392
Name: count, dtype: int64

In [ ]:
print(df['subject'].sample(5).values)

[nan '[Ip-health] Coming soon: the shopping channel run by drug firms'
 "You're In My Thoughts" 'FDA approved on-line pharmacies'
 'Why be an average guy any longer']


In [ ]:
print(df['message'].sample(5).values)

['Hi,\n\nThis is an announcement for a package that has been up on CRAN since March 2006 but was never announced.\n\nThe package is Mchtest - for Monte Carlo hypothesis tests allowing sequential stopping.  The idea is to use the sequential probability ratio test boundaries to stop resampling for a Monte Carlo hypothesis test such as a bootstrap or permutation test. This means that you will take many samples when the p-value is close to the significance level (e.g., 0.05), but you may stop after a much  fewer samples if it becomes clear that the p-value is either very likely to be less than or more than 0.05. \n\nThe details are described in:\n\n\t\tFay, MP, Kim, H-J, and Hachey, M. (2007). "On using Truncated Sequential Probability Ratio Test Boundaries for Monte Carlo Implementation of Hypothesis Tests." (to appear in Journal of Graphical and Computational Statistics).  \n             http://www3.niaid.nih.gov/about/organization/dcr/BRB/staff/michael\n\n\nPlease let me know of any pro

In [ ]:
import re
from bs4 import BeautifulSoup

def clean_email_text(subject, message):
    # Combine subject and message
    text = f"{subject} {message}"

    # 1. Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # 1. Remove content type headers and encoding information
    text = re.sub(r'content-type:.*\n', '', text, flags=re.IGNORECASE)  # Remove content-type lines
    text = re.sub(r'content-transfer-encoding:.*\n', '', text, flags=re.IGNORECASE)  # Remove content-transfer-encoding lines

    # 2. Replace email addresses and URLs
    text = re.sub(r'\S+@\S+', '[EMAIL]', text)              # Replace emails
    text = re.sub(r'http\S+|www\.\S+', '[LINK]', text)      # Replace URLs

    # 3. Remove weird encodings (=20, =\n, \xa0)
    text = re.sub(r'=\n|=20|\xa0', ' ', text)
    text = text.replace('\n', ' ').replace('\r', ' ')

    # 4. Lowercase and remove extra spaces
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
df['clean_email'] = df.apply(lambda row: clean_email_text(row['subject'], row['message']), axis=1)

C:\Users\rtgx2\AppData\Local\Temp\ipykernel_14212\2980855508.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
df['clean_email']

0        generic cialis, branded quality@ content-trans...
1        typo in /debian/readme hi, i've just updated f...
2        authentic viagra charset="iso-8859-1" mega aut...
3        nice talking with ya hey billy, it was really ...
4        or trembling; stomach cramps; trouble in sleep...
                               ...                        
75414    the reply for your request for a job place [le...
75415    the reply for your request for a job place [le...
75416    re: [r] me again, about the horrible documenta...
75417    re: [r] rodbc problem hello, as i wrote i call...
75418    i wanted the desk at his own laws: of the. but...
Name: clean_email, Length: 75419, dtype: object

In [ ]:
print(df['clean_email'].sample(5).values)

["cnn breaking news -- five u.s. soldiers have been killed and three others are missing after an attack 12 miles west of mahmoudiya, iraq, the u.s. military says. watch the latest video now on cnn.com. access at [link] cnn - the most trusted name in news. >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+ it's a side of war reporting rarely seen. michael holmes puts a human face on the casualties of war and shares his personal, behind-the-scenes account of life in iraq through the eyes of a journalist. watch cnn siu: month of mayhem saturday and sunday at 8 and 11 p.m. et. [link] >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+ you have agreed to receive this email from cnn.com as a result of your cnn.com preference settings. to manage your settings, go to: [link] to unsubscribe from the breaking news email alerts, go to: [link] to sign up for additional e-mail products, go to [link] cable news network lp, lllp. one cnn center, atlanta, georgia 30303 (c) 2007 

In [ ]:
print(df.columns)

Index(['label', 'subject', 'email_to', 'email_from', 'message', 'clean_email'], dtype='object')


In [ ]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Get token lengths
df['token_length'] = df['clean_email'].apply(lambda x: len(tokenizer.tokenize(x)))

# Show basic stats
print(df['token_length'].describe())

count     75419.000000
mean       2165.804545
std        6969.610733
min           2.000000
25%         165.000000
50%         357.000000
75%         802.000000
max      685663.000000
Name: token_length, dtype: float64


In [ ]:
print(df.columns)

Index(['label', 'subject', 'email_to', 'email_from', 'message', 'clean_email',
       'token_length'],
      dtype='object')


In [ ]:
print(df['token_length'].sample(5).values)

[ 378   24  296 8323  293]


In [ ]:
df[df['token_length'] > 512].shape[0]

25743

In [ ]:
df['label'].value_counts(normalize=True)

label
1    0.665602
0    0.334398
Name: proportion, dtype: float64

In [ ]:
df[df['token_length'] > 512]['label'].value_counts(normalize=True)

label
1    0.621489
0    0.378511
Name: proportion, dtype: float64

In [ ]:
df[df['token_length'] <= 512]['label'].value_counts(normalize=True)

label
1    0.688461
0    0.311539
Name: proportion, dtype: float64

In [ ]:
def encode_email(text):
    return tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

In [ ]:
sample = df['clean_email'].iloc[0]
encoded = encode_email(sample)

In [ ]:
encoded

{'input_ids': tensor([[  101, 12391,  9915,  6856,  1010, 11180,  3737,  1030,  4180,  1011,
          4651,  1011, 17181,  1024,  1021, 16313,  2079,  2017,  2514,  1996,
          3778,  2000,  4685,  1998,  2025,  4803,  2000,  1996,  6686,  1029,
          1029,  3046,  1058, 24264, 24665,  1037,  1012,  1012,  1012,  1012,
          1012,  2115, 10089,  2097,  2022,  1037,  2518,  1997,  1996,  2627,
          1998,  2017,  2097,  2022,  2067,  2000,  2115,  2214,  2969,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
encoded_pair = tokenizer(
    text='This is the subject.',
    text_pair='This is the body of the email.',
    padding='max_length',
    truncation=True,
    max_length=20,
    return_tensors='pt'
)

print(encoded_pair['input_ids'])
print(encoded_pair['token_type_ids'])

tensor([[  101,  2023,  2003,  1996,  3395,  1012,   102,  2023,  2003,  1996,
          2303,  1997,  1996, 10373,  1012,   102,     0,     0,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])


In [ ]:
encoded_all = tokenizer(
    df['clean_email'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='pt'
)

In [ ]:
encoded_all

{'input_ids': tensor([[  101, 12391,  9915,  ...,     0,     0,     0],
        [  101,  5939,  6873,  ...,     0,     0,     0],
        [  101, 14469,  3081,  ...,     0,     0,     0],
        ...,
        [  101,  2128,  1024,  ...,  2079,  3191,   102],
        [  101,  2128,  1024,  ...,  1035,  1035,   102],
        [  101,  1045,  2359,  ..., 27251,  2000,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
import torch

from torch.utils.data import TensorDataset

labels = torch.tensor(df['label'].values)  # make sure labels are 0 or 1

dataset = TensorDataset(
    encoded_all['input_ids'],
    encoded_all['attention_mask'],
    labels
)

In [ ]:
# Show the first sample (input_ids, attention_mask, label)
input_ids, attention_mask, label = dataset[4]

print("Input IDs:", input_ids)
print("Attention Mask:", attention_mask)
print("Label:", label)

Input IDs: tensor([  101,  2030, 10226,  1025,  4308, 13675, 25167,  1025,  4390,  1999,
         5777,  1025, 11251,  6065,  6192,  1027,  1000,  1011,  1011,  1011,
         1011,  1027,  1035,  2279, 19362,  2102,  1035, 25604,  1035, 25604,
         2050,  1035,  5890,  2278,  2581,  2581, 27717,  2475,  1012,  1039,
         2581, 22932,  2063,  2575,  2094,  2692,  1000,  1011,  1011,  1011,
         1011,  1011,  1011,  1027,  1035,  2279, 19362,  2102,  1035, 25604,
         1035, 25604,  2050,  1035,  5890,  2278,  2581,  2581, 27717,  2475,
         1012,  1039,  2581, 22932,  2063,  2575,  2094,  2692, 25869, 13462,
         1027,  1000,  3645,  1011,  8732,  2487,  1000,  2291,  1000,  1997,
         1996,  2188,  1012,  2009,  2097,  2031,  1996,  9859,  2000,  2022,
         5799,  2521,  2004,  1045,  2113,  1010,  1998,  2054,  1045,  2572,
         2725,  2306,  2009,  1010,  2004,  1037,  2112,  1997,  2009,  2061,
         2007,  4847,  2000,  1996, 13531,  1997,  29

In [ ]:
# Get first sample from dataset
input_ids = dataset[500][0]

# Decode it to readable text
decoded = tokenizer.decode(input_ids, skip_special_tokens=True)

print(decoded)

[ perl # 42378 ] [ todo ] explicit exit from debug _ break ( ) op is not ok # new ticket created by paul cochrane # please include the string : [ perl # 42378 ] # in the subject line of all future correspondence about this issue. # in the file src / ops / debug. ops in the debug _ break ( ) op, there is an explicit exit ( 0 ), which is marked as being not an ok thing to do. this issue needs to be investigated and fixed.


In [ ]:
# Get first sample from dataset
input_ids = dataset[4][0]

# Decode it to readable text
decoded = tokenizer.decode(input_ids, skip_special_tokens=True)

print(decoded)

or trembling ; stomach cramps ; trouble in sleeping ; weakness loose boundary = " - - - - = _ nextpart _ 001 _ 001a _ 01c77a12. c744e6d0 " - - - - - - = _ nextpart _ 001 _ 001a _ 01c77a12. c744e6d0 charset = " windows - 1251 " system " of the home. it will have the capabilities to be linked far as i know, and what i am doing within it, as a part of it so with respect to the affects of technology on society, we have science. ad agencies are cashin g in on its'commerciality and photographs and paint on electronic canvases, it still seems like silence. white out. black out. lights out. it didn't happen, although far from p erfect, especially in that it precludes a vast explanation for this. i can't understand how people can rely so avant - gardes of the art world. additionally, writers and lawyers yet to re ach its full potential. i can only imagine what future computer screen, ho w would our perception of artificial image business apart. there will be t he computerized sign shops and the

In [ ]:
from sklearn.model_selection import train_test_split

# Create indices
indices = list(range(len(labels)))

# Split with stratify
train_idx, val_idx = train_test_split(
    indices,
    test_size=0.2,
    stratify=labels,
    random_state=42
)

In [ ]:
train_dataset = TensorDataset(
    encoded_all['input_ids'][train_idx],
    encoded_all['attention_mask'][train_idx],
    labels[train_idx]
)

val_dataset = TensorDataset(
    encoded_all['input_ids'][val_idx],
    encoded_all['attention_mask'][val_idx],
    labels[val_idx]
)

In [ ]:
from torch.utils.data import DataLoader

# Set your batch size
batch_size = 16  # or 32 depending on your GPU

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.7.0+cu118
True


In [ ]:
from transformers import BertForSequenceClassification

# Load pre-trained BERT with classification head (binary: spam vs not spam)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from tqdm import tqdm  # progress bar

model.train()  # set model to training mode

for batch in tqdm(train_loader):
    input_ids, attention_mask, labels = [x.to(device) for x in batch]

    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels
    )

    loss = outputs.loss
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

100%|██████████████████████████████████████████████████████████████████████████████| 3771/3771 [36:53<00:00,  1.70it/s]


In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

In [ ]:
print(f'Validation accuracy: {correct / total:.10%}')

Validation accuracy: 99.7613365155%


In [ ]:
# Extract labels from val_dataset
val_labels = [label.item() for _, _, label in val_dataset]

# Count how many are spam (1) vs not-spam (0)
from collections import Counter
print(Counter(val_labels))

Counter({1: 10040, 0: 5044})


In [ ]:
model.eval()

for i in range(5):
    # Grab the i-th sample from val_dataset
    input_ids, attention_mask, label = [x[i].unsqueeze(0).to(device) for x in val_dataset.tensors]

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).item()

    decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    print(f"\nEMAIL {i+1}")
    print("Text:", decoded_text)
    print("True Label:", label.item())
    print("Prediction:", prediction)


EMAIL 1
Text: enjoy them all boundary = " - - - - = _ nextpart _ 001 _ 0008 _ 01c77c25. 9c10f660 " - - - - - - = _ nextpart _ 001 _ 0008 _ 01c77c25. 9c10f660 for a girl, " answered jack, who looked down upon nan with condescending = a1 = a1demi subsided. that she " touldn't tuddy, " and saltonstall, jr., retreated precipitately that, the little brown hood slipped through the hedge nearly every for a girl, " answered jack, who looked down upon nan with condescending for a girl, " answered jack, who looked down upon nan with condescending ones i knew of. i was too late, and found the speaker's chair occupied " thank you, sister. i = a1 = a1new houses, she drove about doing various errands for the aunts new houses, she drove about doing various errands for the aunts = a1 = a1are called kittens, and are dear things. i have two, named huz and i forgot all about them. but when i was making my plan, i knew i had a daughter, who is the rightful heir to the throne of the emerald in satisfactio

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')

print(f"Precision: {precision:.2%}")
print(f"Recall:    {recall:.2%}")
print(f"F1 Score:  {f1:.2%}")

Precision: 99.71%
Recall:    99.93%
F1 Score:  99.82%


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

[[ 5015    29]
 [    7 10033]]
